In [ ]:
# import pandas as pd
# import numpy as np
# from sentence_transformers import SentenceTransformer
# from tqdm import tqdm  
# import psycopg2


# # Initialize the model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # Load the DataFrame
# cv_df = pd.read_csv('../data/cv data/cv_data.csv')

# # Replace NaN and non-string values with empty strings
# for column in  ['work', 'education', 'skills_projects']:
#     cv_df[column] = cv_df[column].fillna('').astype(str)

# # Generate embeddings for each column with a progress bar
# tqdm.pandas()

# # Generate embeddings and store them as lists
# cv_df['work_experience_embedding'] = model.encode(cv_df['work'].tolist(), show_progress_bar=True).tolist()
# cv_df['education_embedding'] = model.encode(cv_df['education'].tolist(), show_progress_bar=True).tolist()
# cv_df['skills_embedding'] = model.encode(cv_df['skills_projects'].tolist(), show_progress_bar=True).tolist()

# cv_df.head()


Batches: 100%|██████████| 43/43 [00:14<00:00,  2.96it/s]


,id,category,work,education,skills_projects,work_experience_embedding,education_embedding,skills_embedding
0,10554236,ACCOUNTANT,Position: Accountant\nCompany: Company Name\nS...,Institution: Northern Maine Community College\...,Skill: Accounting\n\nSkill: General Accounting...,"[-0.07590999454259872, -0.051638126373291016, ...","[0.015305997803807259, 0.018881533294916153, 0...","[0.007019722834229469, 0.0014556957175955176, ..."
1,10674770,ACCOUNTANT,Position: Staff Accountant\nCompany: Company N...,Institution: University of North Carolina\nAre...,Skill: Accounting\nLevel: Master\nKeywords: Qu...,"[-0.062130406498909, -0.008351129479706287, -0...","[0.0033015627413988113, 0.010293427854776382, ...","[0.006499188020825386, -0.023113714531064034, ..."
2,11163645,ACCOUNTANT,Position: Accountant\nCompany: Company Name\nS...,"Institution: Martinez Adult Education, Busines...",Skill: Accounting\nLevel: Advanced\nKeywords: ...,"[-0.0742630884051323, 0.004364261869341135, -0...","[0.006215255241841078, 0.003223267151042819, -...","[0.031939759850502014, -0.07590420544147491, -..."
3,11759079,ACCOUNTANT,Position: Senior Accountant\nCompany: Company ...,"Institution: EMORY UNIVERSITY, Goizueta Busine...",Skill: Accounting\nLevel: Senior\nKeywords: ac...,"[-0.017637889832258224, 0.04112307354807854, -...","[0.03224680945277214, -0.029443513602018356, -...","[0.06475291401147842, -0.028756655752658844, -..."
4,12065211,ACCOUNTANT,Position: Senior Accountant\nCompany: Company ...,Institution: TEMPLE UNIVERSITY\nArea of Study:...,Skill: Accounting\nLevel: Senior\nKeywords: Ad...,"[-0.005623675882816315, 0.021937254816293716, ...","[0.013879172503948212, -0.013068466447293758, ...","[0.015567585825920105, -0.024870531633496284, ..."


In [5]:
# # Connect to PostgreSQL
# conn = psycopg2.connect(
#     dbname="vector_db",
#     user="postgres",
#     password="pass123",
#     host="localhost",
#     port="5432"
# )
# cursor = conn.cursor()

# # Insert data into the database
# for _, row in cv_df.iterrows():
#     cursor.execute("""
#         INSERT INTO cv_embeddings (
#             job_id,work_experience_embedding, education_embedding, skills_embedding
#         ) VALUES (%s, %s, %s, %s)
#     """, (
#         row['id'],
#         row['work_experience_embedding'],
#         row['education_embedding'],
#         row['skills_embedding']
#     ))

# # Commit and close connection
# conn.commit()
# cursor.close()
# conn.close()

# print("Embeddings stored successfully!")


Embeddings stored successfully!


In [1]:
import pandas as pd
import psycopg2
from sentence_transformers import SentenceTransformer

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_query_embeddings(work_experience, education, skills):
    return {
        "work_experience_embedding": model.encode(work_experience),
        "education_embedding": model.encode(education),
        "skills_embedding": model.encode(skills)
    }

# Function to insert job data into the database
def insert_job_data_from_df(cv_df, db_config):
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # Fill NaN values with empty strings
    cv_df = cv_df.fillna('')

    for _, job in cv_df.iterrows():
        embeddings = generate_query_embeddings(
            job['work'],
            job['education'],
            job['skills_projects']
        )
        cursor.execute("""
            INSERT INTO cv_embeddings (
                cv_id, work_experience_embedding, education_embedding, skills_embedding
            ) VALUES (%s, %s::vector, %s::vector, %s::vector)
        """, (
            job['id'],
            embeddings['work_experience_embedding'].tolist(),  # Convert to list
            embeddings['education_embedding'].tolist(),        # Convert to list
            embeddings['skills_embedding'].tolist()            # Convert to list
        ))

    conn.commit()
    cursor.close()
    conn.close()
    print("CV data from DataFrame inserted successfully!")

# Load job data from CSV file
cv_df = pd.read_csv('../data/cv data/cv_data.csv')

# Database configuration
db_config = {
    "dbname": "vector_db",
    "user": "postgres",
    "password": "pass123",
    "host": "localhost",
    "port": "5432"
}

# Insert job data from DataFrame
insert_job_data_from_df(cv_df, db_config)


c:\Users\deepa\Downloads\cv matching\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CV data from DataFrame inserted successfully!


In [3]:
def query_top_k_similar(work_experience, education, skills, db_config, weights=None, k=5):
    if weights is None:
        weights = {"work_experience": 1.0, "education": 1.0, "skills": 1.0}

    embeddings = generate_query_embeddings(work_experience, education, skills)

    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # Prepare embeddings as lists for PostgreSQL compatibility
    work_experience_embedding = embeddings['work_experience_embedding'].tolist()
    education_embedding = embeddings['education_embedding'].tolist()
    skills_embedding = embeddings['skills_embedding'].tolist()

    query = """
    SELECT 
        id,
        cv_id,
        (
            (%s * (work_experience_embedding <-> %s::vector)) +
            (%s * (education_embedding <-> %s::vector)) +
            (%s * (skills_embedding <-> %s::vector))
        ) AS combined_distance
    FROM 
        cv_embeddings
    ORDER BY 
        combined_distance ASC  -- Order by ascending distance
    LIMIT %s;
    """

    cursor.execute(query, (
        weights['work_experience'], work_experience_embedding,
        weights['education'], education_embedding,
        weights['skills'], skills_embedding,
        k
    ))

    results = cursor.fetchall()
    cursor.close()
    conn.close()
    return results

# Example usage
top_k = query_top_k_similar(
    work_experience="5 years in data analysis",
    education="Masters in Data Science",
    skills="Python, R, SQL",
    db_config=db_config,
    weights={"work_experience": 1.0, "education": 1.0, "skills": 1.0},
    k=5
)

# Display the results
for result in top_k:
    print(f"ID: {result[0]}, cv_id: {result[1]}, Distance: {result[2]}")


ID: 461, cv_id: 28790806, Distance: 3.1970545841614397
ID: 731, cv_id: 14871762, Distance: 3.3156379380145085
ID: 787, cv_id: 32042584, Distance: 3.3334732097958493
ID: 318, cv_id: 28998957, Distance: 3.361810421035767
ID: 1195, cv_id: 18365443, Distance: 3.365538449446926
